# Lightweight Adaptive Quiz System (LAQS) - Interactive Exploration

This notebook demonstrates how to use the LAQS system programmatically and explore the results interactively.

## 1. Setup and Imports

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import LAQS components
from src import (
    QuestionBank,
    AdaptiveEngine,
    NonAdaptiveEngine,
    SimulatedLearner,
    LearnerPopulation,
    QuizSimulator,
    ExperimentRunner,
    PerformanceTracker,
    VisualizationEngine,
    StatisticalAnalyzer
)

# Configure plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All imports successful!")

## 2. Explore Question Bank

In [ ]:
# Create question bank
qb = QuestionBank(num_questions=100)

# View statistics
stats = qb.get_statistics()
print("Question Bank Statistics:")
print(f"Total Questions: {stats['total_questions']}")
print(f"\nBy Difficulty: {stats['by_difficulty']}")
print(f"\nBy Topic: {stats['by_topic']}")
print(f"\nAverage Expected Time: {stats['avg_expected_time']}")

In [ ]:
# Sample questions
print("Sample Easy Question:")
print(qb.get_question(difficulty='easy'))
print("\nSample Hard Question:")
print(qb.get_question(difficulty='hard'))

## 3. Create and Test Learner Profiles

In [ ]:
# Create different learner types
struggling_learner = SimulatedLearner("struggling", base_ability=0.4, learning_rate=0.05)
average_learner = SimulatedLearner("average", base_ability=0.6, learning_rate=0.05)
advanced_learner = SimulatedLearner("advanced", base_ability=0.85, learning_rate=0.03)

print("Learner Profiles:")
for learner in [struggling_learner, average_learner, advanced_learner]:
    profile = learner.get_profile()
    print(f"\n{profile['learner_id'].upper()}:")
    print(f"  Base Ability: {profile['base_ability']:.2f}")
    print(f"  Learning Rate: {profile['learning_rate']:.3f}")
    print(f"  Speed Factor: {profile['speed_factor']:.2f}")

## 4. Test Adaptive Engine

In [ ]:
# Create adaptive engine
engine = AdaptiveEngine(
    accuracy_threshold_high=0.80,
    accuracy_threshold_low=0.50,
    time_threshold_factor=1.0,
    window_size=5
)

# Simulate high performance
high_performance = [
    {'correct': True, 'time': 15, 'expected_time': 20, 'difficulty': 'easy'},
    {'correct': True, 'time': 18, 'expected_time': 20, 'difficulty': 'easy'},
    {'correct': True, 'time': 16, 'expected_time': 20, 'difficulty': 'easy'},
    {'correct': True, 'time': 17, 'expected_time': 20, 'difficulty': 'easy'},
    {'correct': True, 'time': 19, 'expected_time': 20, 'difficulty': 'easy'},
]

next_diff = engine.get_next_difficulty(high_performance, 'easy')
print(f"After high performance on easy questions: {next_diff}")

# Simulate low performance
low_performance = [
    {'correct': False, 'time': 50, 'expected_time': 40, 'difficulty': 'hard'},
    {'correct': False, 'time': 55, 'expected_time': 40, 'difficulty': 'hard'},
    {'correct': True, 'time': 48, 'expected_time': 40, 'difficulty': 'hard'},
]

next_diff = engine.get_next_difficulty(low_performance, 'hard')
print(f"After low performance on hard questions: {next_diff}")

## 5. Run Mini Experiment

In [ ]:
# Create components
qb = QuestionBank(num_questions=50)
population = LearnerPopulation(num_learners=3)
runner = ExperimentRunner(qb, num_questions_per_session=10)

# Run experiment
print("Running mini experiment...")
results = runner.run_population_experiment(population, num_sessions=2)
print("✓ Experiment complete!")

# Get summary
summary = runner.get_summary_statistics()
print("\nSummary Statistics:")
print(summary)

## 6. Analyze Results

In [ ]:
# Get tracker
tracker = runner.get_tracker()

# Load session data
sessions_df = tracker.get_all_sessions_df()

print("Session Data Sample:")
display(sessions_df.head())

print(f"\nTotal Sessions: {len(sessions_df)}")
print(f"Adaptive Sessions: {len(sessions_df[sessions_df['quiz_type'] == 'adaptive'])}")
print(f"Non-Adaptive Sessions: {len(sessions_df[sessions_df['quiz_type'] == 'non-adaptive'])}")

In [ ]:
# Statistical analysis
analyzer = StatisticalAnalyzer(tracker)

# Descriptive stats
desc_stats = analyzer.compute_descriptive_stats()
print("Descriptive Statistics:")
for quiz_type, stats in desc_stats.items():
    print(f"\n{quiz_type.upper()}:")
    print(f"  Mean Accuracy: {stats['accuracy']['mean']:.4f}")
    print(f"  Mean Mastery: {stats['mastery_index']['mean']:.4f}")

In [ ]:
# T-tests
t_tests = analyzer.perform_t_tests()
print("T-test Results:")
for metric, result in t_tests.items():
    print(f"\n{metric}:")
    print(f"  t-statistic: {result['t_statistic']:.4f}")
    print(f"  p-value: {result['p_value']:.6f}")
    print(f"  Significant: {result['significant']}")

## 7. Custom Visualizations

In [ ]:
# Accuracy comparison
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
sessions_df.boxplot(column='accuracy', by='quiz_type', ax=ax[0])
ax[0].set_title('Accuracy Distribution')
ax[0].set_xlabel('Quiz Type')
ax[0].set_ylabel('Accuracy')
plt.sca(ax[0])
plt.xticks([1, 2], ['Adaptive', 'Non-Adaptive'])

# Bar chart
mean_accuracy = sessions_df.groupby('quiz_type')['accuracy'].mean()
mean_accuracy.plot(kind='bar', ax=ax[1], color=['#2ecc71', '#e74c3c'])
ax[1].set_title('Mean Accuracy Comparison')
ax[1].set_xlabel('Quiz Type')
ax[1].set_ylabel('Mean Accuracy')
ax[1].set_xticklabels(['Adaptive', 'Non-Adaptive'], rotation=0)

plt.tight_layout()
plt.show()

## 8. Load and Analyze Existing Results

In [ ]:
# Load data from CSV files (if they exist)
try:
    sessions = pd.read_csv('data/session_results.csv')
    questions = pd.read_csv('data/question_responses.csv')
    
    print("Loaded existing data:")
    print(f"Sessions: {len(sessions)}")
    print(f"Questions: {len(questions)}")
    
    # Quick analysis
    print("\nQuick Analysis:")
    print(sessions.groupby('quiz_type')[['accuracy', 'mastery_index']].mean())
except FileNotFoundError:
    print("No existing data files found. Run main.py first!")

## 9. Custom Learner Experiment

In [ ]:
# Create a custom learner and run through both quiz types
custom_learner = SimulatedLearner(
    learner_id="custom_001",
    base_ability=0.65,
    learning_rate=0.06,
    speed_factor=1.1,
    consistency=0.85
)

qb = QuestionBank(num_questions=50)
simulator = QuizSimulator(qb)
tracker = PerformanceTracker()

# Run adaptive session
print("Running adaptive quiz...")
adaptive_result = simulator.run_quiz_session(
    custom_learner, 
    num_questions=15, 
    quiz_type='adaptive',
    tracker=tracker,
    session_id='custom_adaptive'
)

# Reset learner
custom_learner.reset()

# Run non-adaptive session
print("Running non-adaptive quiz...")
non_adaptive_result = simulator.run_quiz_session(
    custom_learner,
    num_questions=15,
    quiz_type='non-adaptive',
    tracker=tracker,
    session_id='custom_non_adaptive'
)

# Compare
print("\n=== RESULTS ===")
print(f"\nAdaptive:")
print(f"  Accuracy: {adaptive_result['final_accuracy']:.2%}")
print(f"  Mastery: {adaptive_result['mastery_index']:.3f}")
print(f"  Difficulty Progression: {' → '.join(adaptive_result['difficulty_progression'][:8])}...")

print(f"\nNon-Adaptive:")
print(f"  Accuracy: {non_adaptive_result['final_accuracy']:.2%}")
print(f"  Mastery: {non_adaptive_result['mastery_index']:.3f}")
print(f"  Difficulty: {non_adaptive_result['difficulty_progression'][0]} (constant)")

## 10. Experiment with Different Parameters

In [ ]:
# Test different adaptive thresholds
thresholds = [
    (0.70, 0.40),  # More lenient
    (0.80, 0.50),  # Default
    (0.90, 0.60),  # More strict
]

results_by_threshold = []

for high, low in thresholds:
    engine = AdaptiveEngine(
        accuracy_threshold_high=high,
        accuracy_threshold_low=low
    )
    
    # Simulate performance
    performance = [
        {'correct': np.random.random() > 0.3, 'time': 25, 'expected_time': 30, 'difficulty': 'medium'}
        for _ in range(10)
    ]
    
    # Track difficulty changes
    current_diff = 'medium'
    difficulties = [current_diff]
    
    for i in range(5):
        current_diff = engine.get_next_difficulty(performance[:i+1], current_diff)
        difficulties.append(current_diff)
    
    results_by_threshold.append({
        'thresholds': f"{high}/{low}",
        'progression': difficulties
    })

print("Difficulty Progression by Threshold:")
for result in results_by_threshold:
    print(f"\n{result['thresholds']}: {' → '.join(result['progression'])}")

## Conclusion

This notebook demonstrates:
- How to use LAQS components programmatically
- Create custom learner profiles
- Run experiments with different configurations
- Analyze and visualize results
- Compare adaptive vs non-adaptive approaches

For a full simulation with comprehensive results, run:
```bash
python main.py --full
```